In [86]:
import torch
import torch.nn.functional as F
from sae_lens import SAE
from transformer_lens import HookedTransformer
from bidict import bidict
import numpy as np
import os
from datasets import load_dataset

def get_saes(device, total_layers=26):
    """Loads a series of Sparse Autoencoders (SAEs) for specified layers"""
    saes = []
    print(f"Loading {total_layers} SAEs...")
    for layer in range(total_layers):
        sae, _, _ = SAE.from_pretrained(
            release="gemma-scope-2b-pt-res-canonical",
            sae_id=f"layer_{layer}/width_16k/canonical",
            device=device
        )
        saes.append(sae)
        print(f"Layer {layer} loaded.")
    return saes

def get_all_mmlu_questions(dataset):
    """Returns all questions from the MMLU dataset"""
    all_questions = []
    for i in range(len(dataset)):
        data_point = dataset[i]
        question = data_point['question']
        choices = data_point['choices']
        instruction = "The following is a multiple choice question. Output only a single token corresponding to the right answer (ie: A) \n"
        formatted_question = instruction + f" Question: {question}\n"
        choice_labels = ['A', 'B', 'C', 'D']
        for j, choice in enumerate(choices):
            formatted_question += f"{choice_labels[j]}) {choice}\n"
        formatted_question += "Answer: "
        all_questions.append({
            'text': formatted_question,
            'subject': data_point['subject'],
            'answer': data_point['answer'],
            'choices': data_point['choices'],
            'raw_question': question
        })
    return all_questions
        
def process_question(model, saes, question, feature_bidict, sample_idx=None):
    """
    Performs a forward pass, extracts features and loss for a single MMLU question
    """
    question_text = question['text']
    correct_answer_idx = question['answer']
    choice_labels = ['A', 'B', 'C', 'D']
    total_features = len(feature_bidict)
    feature_vector = np.zeros(total_features, dtype=np.byte)

    with torch.no_grad():
        input_tokens = model.to_tokens(question_text, prepend_bos=True).to(model.cfg.device)
        logits, cache = model.run_with_cache(input_tokens)
        last_token_logits = logits[0, -1, :]
        choice_token_ids = [model.to_tokens(label, prepend_bos=False)[0, 0].item() for label in choice_labels]
        choice_logits = last_token_logits[choice_token_ids]
        output_logits = choice_logits.cpu().numpy()
        loss = F.cross_entropy(choice_logits.unsqueeze(0), torch.tensor([correct_answer_idx]).to(model.cfg.device)).item()
        predicted_choice_idx = torch.argmax(choice_logits).item()
        is_correct = predicted_choice_idx == correct_answer_idx

        print(f"\n--- Sample {sample_idx} ---")
        print(f"Subject: {question['subject']}")
        print(f"Question: {question['raw_question']}")
        print("Choices:")
        for i, choice in enumerate(question['choices']):
            print(f"  {choice_labels[i]}) {choice}")
        print(f"Correct Answer: {choice_labels[correct_answer_idx]} ({question['choices'][correct_answer_idx]})")
        print(f"Model's Answer: {choice_labels[predicted_choice_idx]} ({question['choices'][predicted_choice_idx]})")
        print(f"Result: {'✓ CORRECT' if is_correct else '✗ INCORRECT'}")
        print(f"Loss: {loss:.4f}")

        for layer_idx, sae in enumerate(saes):
            final_token_activations = cache[sae.cfg.hook_name][0, -1, :].unsqueeze(0)
            feature_acts = sae.encode(final_token_activations)
            active_indices = torch.where(feature_acts > 0)[1].cpu().tolist()
            for feature_idx in active_indices:
                global_feature_idx = feature_bidict.get((layer_idx, feature_idx))
                if global_feature_idx is not None:
                    feature_vector[global_feature_idx] = 1

    return feature_vector, is_correct, loss, output_logits
        
def extract_features_and_correctness(model, saes, questions, feature_bidict, output_dir=None):
    """
    Processes questions to get features and correctness, with an option to save data
    """
    all_feature_vectors = []
    all_correctness_labels = []
    all_losses = []
    all_output_logits = []
    for i, question in enumerate(questions):
        feature_vector, is_correct, loss, output_logits = process_question(
            model, saes, question, feature_bidict, sample_idx=i+1
        )
        all_feature_vectors.append(feature_vector)
        all_correctness_labels.append(is_correct)
        all_losses.append(loss)
        all_output_logits.append(output_logits)

    if output_dir:
        features_np = np.array(all_feature_vectors, dtype=np.byte)
        correctness_np = np.array(all_correctness_labels)
        losses_np = np.array(all_losses)
        output_logits_np =  np.array(all_output_logits)
        feature_path = os.path.join(output_dir, f"features.npy")
        correctness_path = os.path.join(output_dir, f"correctness.npy")
        loss_path = os.path.join(output_dir, f"losses.npy")
        output_logits_path = os.path.join(output_dir, f"output_logits.npy")
        np.save(feature_path, features_np)
        np.save(correctness_path, correctness_np)
        np.save(loss_path, losses_np)
        np.save(output_logits_path, output_logits_np)

    return np.array(all_feature_vectors, dtype=np.byte), np.array(all_correctness_labels), np.array(all_losses), np.array(all_output_logits)

def load_or_create_data(model, saes, feature_bidict, mmlu_dataset, output_dir):
    feature_path = os.path.join(output_dir, "features.npy")
    correctness_path = os.path.join(output_dir, "correctness.npy")
    loss_path = os.path.join(output_dir, "losses.npy")
    output_logits_path = os.path.join(output_dir, "output_logits.npy")
    if os.path.exists(feature_path) and os.path.exists(correctness_path) and os.path.exists(loss_path): #and os.path.exists(output_logits_path):
        print(f"Found cached data. Loading from '{output_dir}'...")
        feature_vectors = np.load(feature_path)
        correctness_labels = np.load(correctness_path)
        losses = np.load(loss_path)
        try: 
            output_logits = np.load(output_logits_path)
        except:
            print("output_logits set to None")
            output_logits = None
    else:
        print(f"No cached data found in '{output_dir}'. Generating data from scratch...")
        questions = get_all_mmlu_questions(mmlu_dataset)
        feature_vectors, correctness_labels, losses, output_logits = extract_features_and_correctness(
            model, saes, questions, feature_bidict,
            output_dir=output_dir)
    return feature_vectors, correctness_labels, losses, output_logits

class Node:
    def __init__(self, data=None, split_criteria=None, left=None, right=None):
        self.data = data
        self.split_criteria = split_criteria
        self.left = left
        self.right = right

def find_best_split(objects, losses):
    n_objects = objects.shape[0]
    if n_objects < 2:
        return None
    
    best_info_gain = -1
    best_pos = None
    
    # Calculate current entropy based on losses
    current_entropy = calculate_entropy(losses)
    
    # Try each feature as a potential split
    for feature_idx in range(objects.shape[1]):
        # Split based on feature value
        left_mask = objects[:, feature_idx] == 0
        right_mask = objects[:, feature_idx] == 1
        
        if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
            continue
            
        left_losses = losses[left_mask]
        right_losses = losses[right_mask]
        
        # Calculate weighted entropy after split
        left_weight = len(left_losses) / n_objects
        right_weight = len(right_losses) / n_objects
        weighted_entropy = (left_weight * calculate_entropy(left_losses) + 
                          right_weight * calculate_entropy(right_losses))
        
        # Information gain = current entropy - weighted entropy after split
        info_gain = current_entropy - weighted_entropy
        
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_pos = feature_idx
    
    return best_pos if best_info_gain > 0 else None

def calculate_entropy(losses):
    """Calculate entropy based on loss values (you may want to discretize continuous losses)"""
    if len(losses) == 0:
        return 0
    
    # Option 1: Discretize losses into bins
    # You can adjust the threshold based on your loss distribution
    low_loss = losses < 0.9  # or whatever threshold makes sense
    if len(np.unique(low_loss)) == 1:
        return 0
    
    p_low = np.mean(low_loss)
    p_high = 1 - p_low
    
    entropy = 0
    if p_low > 0:
        entropy -= p_low * np.log2(p_low)
    if p_high > 0:
        entropy -= p_high * np.log2(p_high)
    
    return entropy

def build_tree(objects, losses, current_criteria=None):
    print(f"Current leaf has {len(objects)} objects")
    if current_criteria is None:
        current_criteria = []
    if objects.shape[0] <= 1:
        return Node(data=objects, split_criteria=current_criteria[:])
    split_pos = find_best_split(objects, losses)  # Pass losses here
    if split_pos is None:
        return Node(data=objects, split_criteria=current_criteria[:])
    left_mask = objects[:, split_pos] == 0
    right_mask = ~left_mask
    left_objects = objects[left_mask]
    right_objects = objects[right_mask]
    left_losses = losses[left_mask]
    right_losses = losses[right_mask]
    if left_objects.shape[0] == 0 or right_objects.shape[0] == 0:
        return Node(data=objects, split_criteria=current_criteria[:])
    right_criteria = current_criteria + [[split_pos, 1]]
    return Node(
        split_criteria=current_criteria[:],
        left=build_tree(left_objects, left_losses, current_criteria),
        right=build_tree(right_objects, right_losses, right_criteria)
    )

# Init all variables and load SAEs for all layers
# device = "cuda" if torch.cuda.is_available() else "cpu"
# TEST_OUTPUT_DIR = "cached_test_data_losses"
# os.makedirs(TEST_OUTPUT_DIR, exist_ok=True)
# model = HookedTransformer.from_pretrained("gemma-2-2b", device=device)
# saes = get_saes(device, total_layers=26)
# feature_bidict = bidict()
# global_idx = 0
# for layer_idx, sae in enumerate(saes):
#     for feature_idx in range(sae.cfg.d_sae):
#         feature_bidict[(layer_idx, feature_idx)] = global_idx
#         global_idx += 1
# total_features = len(feature_bidict)
# print(f"Created mapping for {total_features} features across {len(saes)} layers")

# Build tree
MMLU_test_split = load_dataset("cais/mmlu", "all", split='test')
test_feature_vectors, test_correctness_labels, test_losses, test_output_logits = load_or_create_data(model, saes, feature_bidict, MMLU_test_split, TEST_OUTPUT_DIR)
objects_list = test_feature_vectors
objects_np = np.array(objects_list)
tree = build_tree(objects_np, test_losses)

# Testing
def find_leaf(node, datapoint):
    if node.data is not None:
        return node
    split_pos, split_value = node.right.split_criteria[-1]
    
    if datapoint[split_pos] == split_value:
        return find_leaf(node.right, datapoint)
    else:
        return find_leaf(node.left, datapoint)
    
def find_leaf_loss(validation_leaf_prediction, test_feature_vectors, test_losses):
    leaf_data = validation_leaf_prediction.data
    index = [np.where((test_feature_vectors == row).all(axis=1))[0][0] for row in leaf_data][0]
    print(test_losses[index])
    return test_losses[index]

VALIDATION_OUTPUT_DIR = "cached_validation_data_activations_correctness_losses_logits"
os.makedirs(VALIDATION_OUTPUT_DIR, exist_ok=True)
MMLU_validation_split = load_dataset("cais/mmlu", "all", split='validation')
validation_feature_vectors, validation_correctness_labels, validation_losses, output_logits = load_or_create_data(model, saes, feature_bidict, MMLU_validation_split , VALIDATION_OUTPUT_DIR)
validation_leaf_predictions = [find_leaf(tree, validation_feature_vector) for validation_feature_vector in validation_feature_vectors]
validation_loss_predictions = [find_leaf_loss(validation_leaf_prediction, test_feature_vectors, test_losses) for validation_leaf_prediction in validation_leaf_predictions]
low_loss_threshold = 0.9
validation_loss_predictions = np.array(validation_loss_predictions)
confident_mask = validation_loss_predictions < low_loss_threshold
confident_correctness_labels = validation_correctness_labels[confident_mask]
confident_accuracy = np.mean(confident_correctness_labels)

print(f"Coverage of confident predictions: {np.mean(confident_mask):.3f} ({np.sum(confident_mask)}/{len(validation_feature_vectors)} validation samples)")
print(f"Accuracy on confident predictions: {np.mean(confident_correctness_labels):.4f}")
print(f"Model's true validation accuracy: {np.sum(validation_correctness_labels) / len(validation_correctness_labels):.4f}")

Found cached data. Loading from 'cached_test_data_losses'...
output_logits set to None
Current leaf has 14042 objects
Current leaf has 11833 objects
Current leaf has 11355 objects
Current leaf has 8958 objects
Current leaf has 8579 objects
Current leaf has 8368 objects
Current leaf has 8272 objects
Current leaf has 2839 objects
Current leaf has 2195 objects
Current leaf has 2167 objects
Current leaf has 2117 objects
Current leaf has 2010 objects
Current leaf has 1988 objects
Current leaf has 1944 objects
Current leaf has 1725 objects
Current leaf has 1719 objects
Current leaf has 1713 objects
Current leaf has 1531 objects
Current leaf has 1156 objects
Current leaf has 1128 objects
Current leaf has 1094 objects
Current leaf has 1061 objects
Current leaf has 1056 objects
Current leaf has 974 objects
Current leaf has 950 objects
Current leaf has 866 objects
Current leaf has 825 objects
Current leaf has 820 objects
Current leaf has 815 objects
Current leaf has 804 objects
Current leaf has 

KeyboardInterrupt: 

In [107]:
def find_leaf(node, datapoint):
    if node.data is not None:
        return node
    split_pos, split_value = node.right.split_criteria[-1]
    
    if datapoint[split_pos] == split_value:
        return find_leaf(node.right, datapoint)
    else:
        return find_leaf(node.left, datapoint)
    
def find_leaf_loss(validation_leaf_prediction, test_feature_vectors, test_losses):
    leaf_data = validation_leaf_prediction.data
    if len(leaf_data) == 0:
        return None
    target_row = leaf_data[0]
    matches = np.all(test_feature_vectors == target_row, axis=1)
    indices = np.where(matches)[0]
    if len(indices) > 0:
        return test_losses[indices[0]]
    return None

VALIDATION_OUTPUT_DIR = "cached_validation_data_activations_correctness_losses_logits"
os.makedirs(VALIDATION_OUTPUT_DIR, exist_ok=True)
MMLU_validation_split = load_dataset("cais/mmlu", "all", split='validation')
validation_feature_vectors, validation_correctness_labels, validation_losses, output_logits = load_or_create_data(model, saes, feature_bidict, MMLU_validation_split , VALIDATION_OUTPUT_DIR)
validation_leaf_predictions = [find_leaf(tree, validation_feature_vector) for validation_feature_vector in validation_feature_vectors]
validation_loss_predictions = [find_leaf_loss(validation_leaf_prediction, test_feature_vectors, test_losses) for validation_leaf_prediction in validation_leaf_predictions]
low_loss_threshold = 0.9
validation_loss_predictions = np.array(validation_loss_predictions)
confident_mask = validation_loss_predictions < low_loss_threshold
confident_correctness_labels = validation_correctness_labels[confident_mask]
confident_accuracy = np.mean(confident_correctness_labels)

print(f"Coverage of confident predictions: {np.mean(confident_mask):.3f} ({np.sum(confident_mask)}/{len(validation_feature_vectors)} validation samples)")
print(f"Accuracy on confident predictions: {np.mean(confident_correctness_labels):.4f}")
print(f"Model's true validation accuracy: {np.sum(validation_correctness_labels) / len(validation_correctness_labels):.4f}")

Found cached data. Loading from 'cached_validation_data_activations_correctness_losses_logits'...
Coverage of confident predictions: 0.393 (602/1531 validation samples)
Accuracy on confident predictions: 0.5017
Model's true validation accuracy: 0.4161


In [122]:
low_loss_threshold = 0.3
validation_loss_predictions = np.array(validation_loss_predictions)
confident_mask = validation_loss_predictions < low_loss_threshold
confident_correctness_labels = validation_correctness_labels[confident_mask]
confident_accuracy = np.mean(confident_correctness_labels)

print(f"Coverage of confident predictions: {np.mean(confident_mask):.3f} ({np.sum(confident_mask)}/{len(validation_feature_vectors)} validation samples)")
print(f"Accuracy on confident predictions: {np.mean(confident_correctness_labels):.4f}")
print(f"Model's true validation accuracy: {np.sum(validation_correctness_labels) / len(validation_correctness_labels):.4f}")

Coverage of confident predictions: 0.176 (270/1531 validation samples)
Accuracy on confident predictions: 0.6111
Model's true validation accuracy: 0.4161
